In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy.optimize import minimize
import networkx as nx

class SchellingModel:
    def __init__(self, grid_size, similarity_threshold, max_iterations, strategist_population_percentage):
        self.grid_size = grid_size
        self.similarity_threshold = similarity_threshold
        self.max_iterations = max_iterations
        self.strategist_population_percentage = strategist_population_percentage

        self.grid = np.zeros((self.grid_size, self.grid_size))
        self.grid[self.grid == 0] = 3
        self.agent_positions = []

        self.kernel = np.array([[1, 1, 1],
                                [1, 0, 1],
                                [1, 1, 1]], dtype=np.int8)

    def initialize_agents(self, num_agents):
        vacant_spots = [(i, j) for i in range(self.grid_size) for j in range(self.grid_size) if self.grid[i, j] == 3]
        np.random.shuffle(vacant_spots)

        num_strategists = int(num_agents * self.strategist_population_percentage)
        num_schelling = num_agents - num_strategists

        for _ in range(num_strategists):
            position = vacant_spots.pop()
            self.grid[position[0], position[1]] = 1
            self.agent_positions.append(position)

        for _ in range(num_schelling):
            position = vacant_spots.pop()
            self.grid[position[0], position[1]] = 2
            self.agent_positions.append(position)

        plt.figure(figsize=(4,4))
        cmap = mcolors.ListedColormap(['red', 'grey', 'white'])
        col_bounds = [1, 2, 3, 4]
        norm = mcolors.BoundaryNorm(col_bounds, cmap.N)
        plt.figure(figsize=(4,4))
        plt.imshow(self.grid, cmap=cmap, norm=norm)
        plt.show()
        plt.close()

    def get_wrapped_idx(self, idx):
        return idx % self.grid_size

    def wrapped_neighborhood(self, agent_position):
        x, y = agent_position
        neighborhood_indices = [
            (self.get_wrapped_idx(x - 1), self.get_wrapped_idx(y - 1)),
            (self.get_wrapped_idx(x - 1), self.get_wrapped_idx(y)),
            (self.get_wrapped_idx(x - 1), self.get_wrapped_idx(y + 1)),
            (self.get_wrapped_idx(x), self.get_wrapped_idx(y - 1)),
            agent_position,
            (self.get_wrapped_idx(x), self.get_wrapped_idx(y + 1)),
            (self.get_wrapped_idx(x + 1), self.get_wrapped_idx(y - 1)),
            (self.get_wrapped_idx(x + 1), self.get_wrapped_idx(y)),
            (self.get_wrapped_idx(x + 1), self.get_wrapped_idx(y + 1))
        ]

        return neighborhood_indices

    def calculate_similarity(self, agent_position):

        neighborhood_indices = self.wrapped_neighborhood(agent_position)

        neighborhood = self.grid[[int(i) for i, j in neighborhood_indices],
                                 [int(j) for i, j in neighborhood_indices]].reshape(3, 3)

        same_neigh = (neighborhood == 2)*self.kernel
        total_neigh = (neighborhood != 3)*self.kernel
        similarity_ratio = same_neigh.sum() / total_neigh.sum() if total_neigh.sum()>0 else 0

        return similarity_ratio

    def cost_function(self, new_position, agent_position):

        old_neighborhood_indices = self.wrapped_neighborhood(agent_position)
        new_neighborhood_indices = self.wrapped_neighborhood(new_position)

        old_neighborhood = self.grid[[int(i) for i, j in old_neighborhood_indices],
                                     [int(j) for i, j in old_neighborhood_indices]].reshape(3, 3)
        new_neighborhood = self.grid[[int(i) for i, j in new_neighborhood_indices],
                                     [int(j) for i, j in new_neighborhood_indices]].reshape(3, 3)

        diff_strat = np.abs(((old_neighborhood == 1)*self.kernel).sum() - ((new_neighborhood == 1)*self.kernel).sum())
        diff_nonstrat = np.abs(((old_neighborhood == 2)*self.kernel).sum() - ((new_neighborhood == 2)*self.kernel).sum())
        diff_vacant = np.abs(((old_neighborhood == 3)*self.kernel).sum() - ((new_neighborhood == 3)*self.kernel).sum())

        connectivity_difference = diff_strat + diff_nonstrat + diff_vacant

        return -(connectivity_difference)

    def find_groups(self, agent_type=1):

        graph = nx.Graph()  # Create a graph for red agents in this iteration
        agent_positions = {(i, j) for i in range(self.grid_size) for j in range(self.grid_size) if self.grid[i, j] == agent_type}

        for agent_position in agent_positions:

            neighborhood_indices = self.wrapped_neighborhood(agent_position)
            alike_neighbors = [(int(i), int(j)) for i, j in neighborhood_indices
                          if self.grid[int(i), int(j)] == agent_type]

            graph.add_node(agent_position)
            graph.add_edges_from([(agent_position, neighbor) for neighbor in alike_neighbors])

        components = list(nx.connected_components(graph))
        group_sizes = [len(component) for component in components]

        return group_sizes


    def run_simulation(self):
        plt.figure(figsize=(4,4))
        cmap = mcolors.ListedColormap(['red', 'grey', 'white'])
        col_bounds = [1, 2, 3, 4]
        norm = mcolors.BoundaryNorm(col_bounds, cmap.N)
        
        groups_evolution = {}

        for iteration in range(self.max_iterations):
            print(f"Iteration {iteration}")

            np.random.shuffle(self.agent_positions)

            new_agent_positions = set()
            vacant_spots = {(i, j) for i in range(self.grid_size) for j in range(self.grid_size) if self.grid[i, j] == 3}

            for agent_position in self.agent_positions:
              
                vacant_spots = {(i, j) for i in range(self.grid_size) for j in range(self.grid_size) if self.grid[i, j] == 3}
                if agent_position not in vacant_spots:

                  new_agent_positions.add(agent_position)

                  if self.grid[agent_position[0], agent_position[1]] == 1:  # Strategists

                      result = minimize(
                          self.cost_function,
                          x0=list(vacant_spots)[np.random.choice(len(vacant_spots))],
                          args=(agent_position,),
                          bounds=[(0, self.grid_size), (0, self.grid_size)]
                      )
                      new_position = tuple(map(int, result.x))
                      self.grid[new_position[0], new_position[1]] = 1     # move strategist to new position
                      self.grid[agent_position[0], agent_position[1]] = 3 # vacate the old position

                      new_agent_positions.add(new_position)
                      new_agent_positions.remove(agent_position)

                      vacant_spots.add(agent_position)
                      vacant_spots.remove(new_position)


                  elif self.grid[agent_position[0], agent_position[1]] == 2: # Non-strategists

                      similarity = self.calculate_similarity(agent_position)

                      if similarity < self.similarity_threshold:
                        print(similarity, "<", self.similarity_threshold, agent_position)

                        new_position = list(vacant_spots)[np.random.choice(len(vacant_spots))]
                        self.grid[new_position[0], new_position[1]] = 2       # move non-strategist to new position
                        self.grid[agent_position[0], agent_position[1]] = 3   # vacate old position

                        new_agent_positions.add(new_position)
                        new_agent_positions.remove(agent_position)

                        vacant_spots.add(agent_position)
                        vacant_spots.remove(new_position)

            self.agent_positions = list(new_agent_positions)

            plt.imshow(self.grid, cmap=cmap, norm=norm)
            plt.title(f"Iteration {iteration}")
            plt.show()

            groups = self.find_groups(agent_type=1)
            groups_evolution[iteration] = groups
        
        return groups_evolution


if __name__ == "__main__":
    grid_size = 10
    similarity_threshold = 0.5
    max_iterations = 1000
    strategist_population_percentage = 0.2

    model = SchellingModel(grid_size, similarity_threshold, max_iterations, strategist_population_percentage)
    model.initialize_agents(num_agents=50)
    groups_evolution = model.run_simulation()

In [ ]:
max_groups = {it:max(group_sizes) for it,group_sizes in groups_evolution.items()}
plt.figure(figsize=(7, 5))
plt.plot(list(max_groups.values()))